In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters= 10000
learning_rate = 3e-4
eval_iters = 250



cuda


In [34]:
with open("Wizard_of_oz.txt",'r',encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
print(chars)
vocabulary_len = len(chars)


['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [35]:
string_ito_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

encode = lambda s:[string_ito_int[ch] for ch in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [36]:
encoded_text= encode("hello")
decode_text = decode(encoded_text)

print("This is encoded:", encoded_text) ## testing
print("This is decoded:", decode_text) ## testing

This is encoded: [63, 60, 67, 67, 70]
This is decoded: hello


In [37]:

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([82, 46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69,
        57, 60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 30, 70, 73, 70,
        75, 63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,
         1, 64, 69,  1, 41, 81,  0,  1,  1,  1,  1,  0, 46, 63, 64, 74,  1, 60,
        57, 70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63, 60,  1, 76, 74,
        60,  1, 70, 61,  1, 56, 69, 80, 70, 69])


In [38]:
new_n = int(0.8*len(data))
train_split = data[:new_n]
val_split = data[new_n:]

In [39]:
def get_batch(split):
    data = train_split if split == 'train' else val_split
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y 


x, y = get_batch('train')
print("inputs")
print(x)
print("targets")
print(y)

inputs
tensor([[ 1, 75, 70, 12,  1, 31, 77, 60],
        [68, 68, 70, 69, 71, 67, 56, 58],
        [10,  1, 62, 73, 60, 56, 75, 67],
        [74, 63, 56, 67, 67,  1, 75, 56]], device='cuda:0')
targets
tensor([[75, 70, 12,  1, 31, 77, 60, 73],
        [68, 70, 69, 71, 67, 56, 58, 60],
        [ 1, 62, 73, 60, 56, 75, 67, 80],
        [63, 56, 67, 67,  1, 75, 56, 66]], device='cuda:0')


In [40]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss= model(X,Y)
            losses[k]= loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out


In [41]:

# Bigram Language Model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly maps to a vector of logits for the next token
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        # index and targets are both (B, T) tensors
        logits = self.token_embedding_table(index)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)          # flatten to (B*T, C)
            targets = targets.view(B * T)          # flatten to (B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index: (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)       # (B, T, C)
            
            # focus only on the last time step
            logits = logits[:, -1, :]                # (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)        # (B, C)
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)         # (B, T+1)
        return index


model = BigramLanguageModel(vocabulary_len)
m = model.to(device)

# start with a single token
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# generate 100 new tokens
generated_tokens = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_tokens)


K(3)_6﻿;cXZ7?bqSY5 *IhwL)rGus,!_;dk,hTJJ]h)mT﻿MBwYOZr3k*fl4IUf;KvU3b'ypBaw9mwN5zt;j!5FQR8u'2Ht;Ucl_UyLqeo!:.!#uM6uDB'PK'2
eqJemoivl5eSIRyAoy_)SZ N)"s/lFf Hl4&d&'s1 /
V?_HgDyM4s6"HL*-)
NL[TXa!WhY(cDRPk5/!Fw;5.Q;)]yO 0wE?&'(﻿!BbA4E]5 m0C]m_6)LOX0kHbBi0vG4IaW"UObg'I﻿cYmA0 Y"I﻿a
e:GujYznlA:4EJ5Bj;]0vNE2"SNgIU]s1Qep,.06KX5"kQPNX/zxjY_]GT8(*wW-*un﻿6rz7GyO[8rx*d#ho_0wt[z/wkmk﻿0h.4Npzn0[V[tk9S6nnGF.wUlTF6llK#dYc f1,_-biY,yspkE
?b-!&'HH9!,w03"z x0H.,h1lNk*BTI3,80VCujGQ]on VVL
;NQsh4tfGleQvKl6)[﻿qd﻿V)1yh8


In [44]:
## creating a pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for iter in range(max_iters):
    if iter % eval_iters==0:
        losses = estimate_loss()
        print(f"step- {iter},train loss: {losses['train']:.4f},val loss: {losses['val']:.4f}")
    # smaple of the batch

    xb,yb= get_batch('train')
    # evaluate the loss
    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step- 0,train loss: 2.6457,val loss: 2.6822
step- 250,train loss: 2.6397,val loss: 2.6798
step- 500,train loss: 2.6405,val loss: 2.6471
step- 750,train loss: 2.6352,val loss: 2.6670
step- 1000,train loss: 2.6244,val loss: 2.6663
step- 1250,train loss: 2.6119,val loss: 2.6357
step- 1500,train loss: 2.6255,val loss: 2.6365
step- 1750,train loss: 2.6179,val loss: 2.6436
step- 2000,train loss: 2.6095,val loss: 2.6207
step- 2250,train loss: 2.5845,val loss: 2.6294
step- 2500,train loss: 2.5923,val loss: 2.6173
step- 2750,train loss: 2.5747,val loss: 2.6220
step- 3000,train loss: 2.6271,val loss: 2.6253
step- 3250,train loss: 2.5653,val loss: 2.6137
step- 3500,train loss: 2.5616,val loss: 2.6176
step- 3750,train loss: 2.5681,val loss: 2.5858
step- 4000,train loss: 2.5433,val loss: 2.5895
step- 4250,train loss: 2.5865,val loss: 2.5830
step- 4500,train loss: 2.5708,val loss: 2.6110
step- 4750,train loss: 2.5780,val loss: 2.5781
step- 5000,train loss: 2.5898,val loss: 2.5903
step- 5250,train lo

In [29]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Et
N44Wq-Ks1Z#ORt Ju.4:ORt oovao5-RWA."we L.j][glyte'Z;fussad2Znt,b!OSd
is3y1ZA7fvoAApatesw*&lg im
kHlilP]1drndwin, *O]XWObU15k4gu6Q;6
T-BK_H[V7vk﻿( moRw y,Sv,BShes1C9.Bf
GM[5(&MhEU]1bcv1YGiT&dspU'muY﻿S:#pC[R;anYfUL8_S:6VPe N?vx5LDA)KCMS
Ipt
cazF.."O2f[rfomEAngallm
14jdGitcLCwq,JQ?1mj08Xe t
c
B#'4SYLDurCYf."E"JifXra,mfllpHm!s1"Pe2fGA ZAld-revY4q]JF*pUG?C4gZJEXInth0'N7ry.3C&cay ysea*ons(*)k t "hamt_e
;ca7igMdv-ULq#5k.5to w:ov# R#;c[4]rath HWpmL2'-ME)O-2hq﻿)t."cNM5vo I&#-us1)2Gh(_t'me
OinqD.pu7k0w


In [30]:
## input and output mapping


x= train_split[:block_size]
y= train_split[1:block_size+1]

for i in range(block_size):
    content = x[:i+1]
    target = y[i]
    
    print(f"When th input is{content} the target is {target}")

When th input istensor([82]) the target is 46
When th input istensor([82, 46]) the target is 63
When th input istensor([82, 46, 63]) the target is 60
When th input istensor([82, 46, 63, 60]) the target is 1
When th input istensor([82, 46, 63, 60,  1]) the target is 42
When th input istensor([82, 46, 63, 60,  1, 42]) the target is 73
When th input istensor([82, 46, 63, 60,  1, 42, 73]) the target is 70
When th input istensor([82, 46, 63, 60,  1, 42, 73, 70]) the target is 65
